In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exl-training/EQ_2021_Data_Sample.csv
/kaggle/input/exl-test/EQ-2021-Validation-Data-Sample.csv
/kaggle/input/testt-exl/EQ-2021-Validation-Data-Sample.csv


In [2]:
import sklearn
from sklearn.model_selection import train_test_split 
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
data = pd.read_csv("/kaggle/input/exl-training/EQ_2021_Data_Sample.csv")
data.head(2)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (31,32,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,countyFIPS,stateFIPS,date,confirmed_cases,deaths,google_mobility_retail_and_recreation,google_mobility_grocery_and_pharmacy,google_mobility_parks,google_mobility_transit_stations,google_mobility_workplaces,...,total_adult_patients_hospitalized_confirmed_covid_SD,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD,total_pediatric_patients_hospitalized_confirmed_covid_SD,inpatient_beds_utilization_SD,inpatient_beds_utilization_numerator_SD,inpatient_beds_utilization_denominator_SD,percent_of_inpatients_with_covid_SD,inpatient_bed_covid_utilization_SD,adult_icu_bed_covid_utilization_SD,adult_icu_bed_utilization_SD
0,1001,1,2020-01-22,0,0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003,1,2020-01-22,0,0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data['date'] = pd.to_datetime(data['date'])    
data['date_delta'] = (data['date'] - data['date'].min())
data["date_delta"] = data["date_delta"].astype('timedelta64[D]')

In [5]:
numeric_columns = data.select_dtypes(include=['float64','int64']).columns 
object_columns = data.select_dtypes(include=['object']).columns 
print('\nnumeric columns:\n', numeric_columns) 
print('\nobject columns:\n', object_columns)


numeric columns:
 Index(['countyFIPS', 'stateFIPS', 'confirmed_cases', 'deaths',
       'google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy', 'google_mobility_parks',
       'google_mobility_transit_stations', 'google_mobility_workplaces',
       'google_mobility_residential', 'apple_mobility_driving',
       'apple_mobility_transit', 'apple_mobility_walking',
       'Electricity_Sales_ Revenue_Residential ($ x1000)',
       'Electricity_Sales_ Sales_Residential (MWHr)',
       'Electricity_Sales_ Customers_Count_Residential',
       'Electricity_Sales_ Revenue_Commercial ($ x1000)',
       'Electricity_Sales_ Sales_Commercial (MWHr)',
       'Electricity_Sales_ Customers_Count_Commercial',
       'Electricity_Sales_ Revenue_Industrial ($ x1000)',
       'Electricity_Sales_ Sales_Industrial (MWHr)',
       'Electricity_Sales_ Customers_Count_Industrial',
       'Electricity_Sales_ Revenue_Transportation ($ x1000)',
       'Electricity_Sales_ Sales_Tra

In [6]:
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_data = pd.DataFrame({'column_name': data.columns,
                                 'percent_missing': percent_missing})
missing_value_data.sort_values('percent_missing', inplace=True)

In [7]:
missing_value_data

,column_name,percent_missing
countyFIPS,countyFIPS,0.000000
new_test_count,new_test_count,0.000000
new_test_rate,new_test_rate,0.000000
test_rate,test_rate,0.000000
test_count,test_count,0.000000
...,...,...
google_mobility_transit_stations,google_mobility_transit_stations,72.979671
google_mobility_parks,google_mobility_parks,83.303275
YoY_Reopened_Seated_Diner_Data,YoY_Reopened_Seated_Diner_Data,85.917626
apple_mobility_walking,apple_mobility_walking,87.885533


In [8]:
data.drop("apple_mobility_transit", axis=1, inplace = True)
print(data.isnull().sum().sum())
data.fillna(data.median(),inplace=True)
print(data.isnull().sum().sum())

19104682


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


0


In [9]:
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='pearson', min_periods=1)
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out
beta = data.drop(["countyFIPS","stateFIPS","date","confirmed_cases","deaths",'date_delta'], axis=1)
beta = trimm_correlated(beta, .9)
beta.shape

(1179375, 43)

In [10]:
temp = data[['google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy', 'google_mobility_parks',
       'google_mobility_transit_stations', 'google_mobility_workplaces',
       'google_mobility_residential', 'apple_mobility_driving',
       'apple_mobility_walking',
       'Electricity_Sales_ Revenue_Residential ($ x1000)',
       'Electricity_Sales_ Revenue_Industrial ($ x1000)',
       'Electricity_Sales_ Revenue_Transportation ($ x1000)',
       'Electricity_Sales_ Customers_Count_Transportation',
       'DOMESTIC_Air_Passengers', 'YoY_Reopened_Seated_Diner_Data',
       'C_PCTPOVALL_2019', 'C_MEDHHINC_2019', 'C_TOT_POP', 'C_MinorityPCT',
       'C_BlackPCT', 'C_HispanicPCT', 'C_M_Unemployment_Rate_corr',
       'S_D_dly_new_test', 'S_D_cummulative_test',
       'initclaims_count_regular_cw', 'initclaims_rate_regular_cw',
       'spend_all_cd', 'merchants_all_cd', 'revenue_all_cd',
       'critical_staffing_shortage_today_yes_SD', 'hospital_onset_covid_SD',
       'hospital_onset_covid_coverage_SD', 'inpatient_beds_SD',
       'previous_day_admission_adult_covid_confirmed_SD',
       'previous_day_admission_adult_covid_suspected_SD',
       'previous_day_admission_pediatric_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_suspected_SD',
       'staffed_adult_icu_bed_occupancy_SD',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_covid_SD',
       'inpatient_beds_utilization_SD', 'percent_of_inpatients_with_covid_SD',
       'adult_icu_bed_covid_utilization_SD', 'adult_icu_bed_utilization_SD',"countyFIPS","stateFIPS","date_delta","confirmed_cases","deaths"]]

In [11]:
gamma = temp.drop("deaths", axis=1)

In [12]:
y_res = gamma['confirmed_cases']
x_res = gamma.drop(['confirmed_cases'], axis=1, inplace=False)

In [13]:
model = xgb.XGBRegressor(tree_method = 'gpu_hist', gpu_id=0, objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 15, alpha = 10, n_estimators = 100)

In [14]:
xtrain, xtest, ytrain, ytest = train_test_split(x_res, y_res, test_size=0.15)

In [15]:
modelhist = model.fit(xtrain,ytrain)

In [16]:
model.score(xtest,ytest)

0.9939050800199364

In [17]:
ypred = model.predict(xtest)
ypred

array([ 20.395906 ,  -4.635296 , 829.4699   , ..., 451.62805  ,
         1.4803442,  -1.2549728], dtype=float32)

In [18]:
model.score(xtest,ypred)

1.0

In [19]:
model.score(xtrain,ytrain)

0.9999086340051471

In [20]:
test = pd.read_csv("/kaggle/input/testt-exl/EQ-2021-Validation-Data-Sample.csv")
test.head(2)

,countyFIPS,stateFIPS,date,google_mobility_retail_and_recreation,google_mobility_grocery_and_pharmacy,google_mobility_parks,google_mobility_transit_stations,google_mobility_workplaces,google_mobility_residential,apple_mobility_driving,...,total_adult_patients_hospitalized_confirmed_covid_SD,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD,total_pediatric_patients_hospitalized_confirmed_covid_SD,inpatient_beds_utilization_SD,inpatient_beds_utilization_numerator_SD,inpatient_beds_utilization_denominator_SD,percent_of_inpatients_with_covid_SD,inpatient_bed_covid_utilization_SD,adult_icu_bed_covid_utilization_SD,adult_icu_bed_utilization_SD
0,1001,1,2/1/2021,-20.0,-5.0,NaN,NaN,-19.0,7.0,144.74,...,1887,26,16,0.745088,11262,15115,0.187557,0.140003,0.377079,0.87056
1,1003,1,2/1/2021,-18.0,-5.0,-18.0,-16.0,-22.0,5.0,129.11,...,1887,26,16,0.745088,11262,15115,0.187557,0.140003,0.377079,0.87056


In [21]:
test['date'] = pd.to_datetime(data['date'])    
test['date_delta'] = (test['date'] - test['date'].min())
test["date_delta"] = test["date_delta"].astype('timedelta64[D]')

In [22]:
test.columns

Index(['countyFIPS', 'stateFIPS', 'date',
       'google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy', 'google_mobility_parks',
       'google_mobility_transit_stations', 'google_mobility_workplaces',
       'google_mobility_residential', 'apple_mobility_driving',
       'apple_mobility_transit', 'apple_mobility_walking',
       'Electricity_Sales_ Revenue_Residential ($ x1000)',
       'Electricity_Sales_Sales_Residential(MWHr)',
       'Electricity_Sales_Customers_Count_Residential',
       'Electricity_Sales_Revenue_Commercial($ x1000)',
       'Electricity_Sales_Sales_Commercial(MWHr)',
       'Electricity_Sales_Customers_Count_Commercial',
       'Electricity_Sales_ Revenue_Industrial ($ x1000)',
       'Electricity_Sales_Sales_Industrial(MWHr)',
       'Electricity_Sales_ Customers_Count_Transportation',
       'Electricity_Sales_ Revenue_Transportation ($ x1000)',
       'Electricity_Sales_Sales_Transportation(MWHr)',
       'Electricity_Sales_C

In [23]:
test_data = test[['google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy', 'google_mobility_parks',
       'google_mobility_transit_stations', 'google_mobility_workplaces',
       'google_mobility_residential', 'apple_mobility_driving',
       'apple_mobility_walking',
       'Electricity_Sales_ Revenue_Residential ($ x1000)',
       'Electricity_Sales_ Revenue_Industrial ($ x1000)',
       'Electricity_Sales_ Revenue_Transportation ($ x1000)',
       'Electricity_Sales_ Customers_Count_Transportation',
       'DOMESTIC_Air_Passengers', 'YoY_Reopened_Seated_Diner_Data',
       'C_PCTPOVALL_2019', 'C_MEDHHINC_2019', 'C_TOT_POP', 'C_MinorityPCT',
       'C_BlackPCT', 'C_HispanicPCT', 'C_M_Unemployment_Rate_corr',
       'S_D_dly_new_test', 'S_D_cummulative_test',
       'initclaims_count_regular_cw', 'initclaims_rate_regular_cw',
       'spend_all_cd', 'merchants_all_cd', 'revenue_all_cd',
       'critical_staffing_shortage_today_yes_SD', 'hospital_onset_covid_SD',
       'hospital_onset_covid_coverage_SD', 'inpatient_beds_SD',
       'previous_day_admission_adult_covid_confirmed_SD',
       'previous_day_admission_adult_covid_suspected_SD',
       'previous_day_admission_pediatric_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_suspected_SD',
       'staffed_adult_icu_bed_occupancy_SD',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_covid_SD',
       'inpatient_beds_utilization_SD', 'percent_of_inpatients_with_covid_SD',
       'adult_icu_bed_covid_utilization_SD', 'adult_icu_bed_utilization_SD',"countyFIPS","stateFIPS","date_delta"]]

In [24]:
predictions = model.predict(test_data)
predictions

array([ 3261.4836,  4767.5513,  5837.0986, ..., 10639.998 , 23276.889 ,
        9064.207 ], dtype=float32)

In [25]:
predictions_values = np.around(predictions)
predictions_values = predictions_values.astype(int)
predictions_values

array([ 3261,  4768,  5837, ..., 10640, 23277,  9064])

In [26]:
len(predictions_values)

47130

In [27]:
pred = pd.DataFrame(data=predictions_values, columns=["Confirmed_cases"])
pred.head(5)

,Confirmed_cases
0,3261
1,4768
2,5837
3,5477
4,3149


In [28]:
preds_decimal = pd.DataFrame(data=predictions, columns=["Confirmed_cases"])
preds_decimal.head(5)

,Confirmed_cases
0,3261.483643
1,4767.551270
2,5837.098633
3,5477.498535
4,3148.685547


In [29]:
datatest = pd.read_csv("/kaggle/input/testt-exl/EQ-2021-Validation-Data-Sample.csv")
datatest.head(2)

,countyFIPS,stateFIPS,date,google_mobility_retail_and_recreation,google_mobility_grocery_and_pharmacy,google_mobility_parks,google_mobility_transit_stations,google_mobility_workplaces,google_mobility_residential,apple_mobility_driving,...,total_adult_patients_hospitalized_confirmed_covid_SD,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD,total_pediatric_patients_hospitalized_confirmed_covid_SD,inpatient_beds_utilization_SD,inpatient_beds_utilization_numerator_SD,inpatient_beds_utilization_denominator_SD,percent_of_inpatients_with_covid_SD,inpatient_bed_covid_utilization_SD,adult_icu_bed_covid_utilization_SD,adult_icu_bed_utilization_SD
0,1001,1,2/1/2021,-20.0,-5.0,NaN,NaN,-19.0,7.0,144.74,...,1887,26,16,0.745088,11262,15115,0.187557,0.140003,0.377079,0.87056
1,1003,1,2/1/2021,-18.0,-5.0,-18.0,-16.0,-22.0,5.0,129.11,...,1887,26,16,0.745088,11262,15115,0.187557,0.140003,0.377079,0.87056


In [30]:
zeta = datatest[["countyFIPS","stateFIPS","date"]]
zeta.head(2)

,countyFIPS,stateFIPS,date
0,1001,1,2/1/2021
1,1003,1,2/1/2021


In [31]:
finalresult = pd.concat([zeta, pred], axis=1)
finalresult.head(4)

,countyFIPS,stateFIPS,date,Confirmed_cases
0,1001,1,2/1/2021,3261
1,1003,1,2/1/2021,4768
2,1005,1,2/1/2021,5837
3,1007,1,2/1/2021,5477


In [32]:
finalresult.index += 1
finalresult.head(4)

,countyFIPS,stateFIPS,date,Confirmed_cases
1,1001,1,2/1/2021,3261
2,1003,1,2/1/2021,4768
3,1005,1,2/1/2021,5837
4,1007,1,2/1/2021,5477


In [33]:
finalresult.to_csv('finalresult.csv', index = True)

In [34]:
finalresult_decimal = pd.concat([zeta, preds_decimal], axis=1)
finalresult_decimal.head(4)

,countyFIPS,stateFIPS,date,Confirmed_cases
0,1001,1,2/1/2021,3261.483643
1,1003,1,2/1/2021,4767.551270
2,1005,1,2/1/2021,5837.098633
3,1007,1,2/1/2021,5477.498535


In [35]:
finalresult_decimal.index += 1
finalresult_decimal.head(4)

,countyFIPS,stateFIPS,date,Confirmed_cases
1,1001,1,2/1/2021,3261.483643
2,1003,1,2/1/2021,4767.551270
3,1005,1,2/1/2021,5837.098633
4,1007,1,2/1/2021,5477.498535


In [36]:
finalresult_decimal.to_csv('finalresult_decimal.csv', index = True)